In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from nltk.corpus import stopwords
from textblob import TextBlob
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from nltk.stem import PorterStemmer
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from scipy.sparse import hstack
import matplotlib.pyplot as plt
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.sparse import hstack
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, make_scorer, accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample
from scipy.sparse import hstack
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

## baseline model

In [18]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Load the data
df = pd.read_csv('data_cleaned.csv')

def process_column_rf(column_name, data):
    """Function to process data using RandomForest."""
    data_cleaned = data.dropna(subset=[column_name])
    y = data_cleaned['Article_Gender']
    
    results = {}
    
    # Vectorize using unigrams & bigram
    tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
    X = tfidf_vectorizer.fit_transform(data_cleaned[column_name])
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train RandomForest without cross-validation
    rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', oob_score=True, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    # Get scores for Female
    results['precision_female'] = precision_score(y_test, y_pred, pos_label='Female', zero_division=0)
    results['recall_female'] = recall_score(y_test, y_pred, pos_label='Female', zero_division=0)
    results['f1_score_female'] = f1_score(y_test, y_pred, pos_label='Female', zero_division=0)
    
    # Get scores for Male
    results['precision_male'] = precision_score(y_test, y_pred, pos_label='Male', zero_division=0)
    results['recall_male'] = recall_score(y_test, y_pred, pos_label='Male', zero_division=0)
    results['f1_score_male'] = f1_score(y_test, y_pred, pos_label='Male', zero_division=0)

    # Accuracy score
    results['accuracy_score'] = accuracy_score(y_test, y_pred)

    return results

# Process 'Intro_Cleaned' column without dummies
results_intro_cleaned = process_column_rf('Intro_Cleaned', df)

for metric, score in results_intro_cleaned.items():
    print(f"Results for 'Intro_Cleaned' without dummies using {metric}:")
    print(f"Score: {score}\n")

Results for 'Intro_Cleaned' without dummies using precision_female:
Score: 0.6785714285714286

Results for 'Intro_Cleaned' without dummies using recall_female:
Score: 0.04068522483940043

Results for 'Intro_Cleaned' without dummies using f1_score_female:
Score: 0.07676767676767678

Results for 'Intro_Cleaned' without dummies using precision_male:
Score: 0.8416401555319901

Results for 'Intro_Cleaned' without dummies using recall_male:
Score: 0.996234309623431

Results for 'Intro_Cleaned' without dummies using f1_score_male:
Score: 0.9124353324391646

Results for 'Intro_Cleaned' without dummies using accuracy_score:
Score: 0.840042002100105



## with cross validation & weights

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Load the data
df = pd.read_csv('data_cleaned.csv')

def process_column_rf(column_name, data, top_n=20):
    """Function to process data using RandomForest."""
    data_cleaned = data.dropna(subset=[column_name])
    y = data_cleaned['Article_Gender']
    
    results = {}
    
    # Vectorize using unigrams & bigram
    tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
    X = tfidf_vectorizer.fit_transform(data_cleaned[column_name])
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf_params = {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Loop through weights for female from 1 to 11
    for weight in range(1, 12):
        class_weights = {'Male': 1, 'Female': weight}
        
        grid_search_rf = GridSearchCV(RandomForestClassifier(class_weight=class_weights, oob_score=True, random_state=42),
                                      rf_params,
                                      cv=5,
                                      scoring='accuracy',
                                      n_jobs=-1)
        grid_search_rf.fit(X_train, y_train)
        best_params_rf = grid_search_rf.best_params_
        
        # Use the best estimator for predictions
        best_rf = grid_search_rf.best_estimator_
        y_pred = best_rf.predict(X_test)
        
        # Calculate scores for female
        precision_female = precision_score(y_test, y_pred, pos_label='Female', zero_division=0)
        recall_female = recall_score(y_test, y_pred, pos_label='Female', zero_division=0)
        f1_female = f1_score(y_test, y_pred, pos_label='Female', zero_division=0)
        
        # Calculate scores for male
        precision_male = precision_score(y_test, y_pred, pos_label='Male', zero_division=0)
        recall_male = recall_score(y_test, y_pred, pos_label='Male', zero_division=0)
        f1_male = f1_score(y_test, y_pred, pos_label='Male', zero_division=0)

        # Accuracy
        accuracy = accuracy_score(y_test, y_pred)

        # Store the results for this weight
        results[f"weight_female_{weight}"] = {
            'precision_female': precision_female,
            'recall_female': recall_female,
            'f1_score_female': f1_female,
            'precision_male': precision_male,
            'recall_male': recall_male,
            'f1_score_male': f1_male,
            'accuracy': accuracy
        }

    return results

# Process 'Intro_Cleaned' column without dummies
results_intro_cleaned = process_column_rf('Intro_Cleaned', df)

for weight, metrics in results_intro_cleaned.items():
    print(f"Results for 'Intro_Cleaned' without dummies using {weight}:")
    for metric, score in metrics.items():
        print(f"{metric}: {score}")
    print("\n")

## Validation curve

In [16]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('data_cleaned.csv')

data_cleaned = df.dropna(subset=['Intro_Cleaned'])
y = data_cleaned['Article_Gender']
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X = tfidf_vectorizer.fit_transform(data_cleaned['Intro_Cleaned'])
weights = range(1, 12)

# Using F1 score as the scoring metric
train_scores, test_scores = validation_curve(
    RandomForestClassifier(n_estimators=100, random_state=42),
    X, y, param_name="class_weight", param_range=[{'Male': 1, 'Female': w} for w in weights],
    scoring=make_scorer(f1_score, pos_label='Female', zero_division=0), cv=5)

train_scores_mean = train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
test_scores_mean = test_scores.mean(axis=1)
test_scores_std = test_scores.std(axis=1)

plt.figure(figsize=(10,6))
plt.title("Validation Curve with RandomForest for F1 Score")
plt.xlabel("Weight for Female")
plt.ylabel("F1 Score")
plt.ylim(0.5, 1.1)
plt.plot(weights, train_scores_mean, label="Training score", color="r")
plt.fill_between(weights, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.2, color="r")
plt.plot(weights, test_scores_mean, label="Cross-validation score", color="g")
plt.fill_between(weights, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.2, color="g")
plt.legend(loc="best")
plt.show()

KeyboardInterrupt: 